In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
import bz2

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor

import sys
sys.path.append("..")
from meta_stream import MetaStream

In [ ]:
df_rows = 50000
file = bz2.open("../../data/DataExpo2009/2008.csv.bz2", 'r')
df = pd.read_csv(file, nrows=df_rows)
df = df[['DepTime', 'CRSDepTime', 'CRSArrTime', 'ArrDelay', 'Distance']]
df = df.dropna(axis=0, how='any')

In [ ]:
# NOTE: list of regression algorithms
models =    [
            SVR(),
            RandomForestRegressor(random_state=42),
            LinearRegression(),
            Ridge(),
            GradientBoostingRegressor(random_state=42),
            DecisionTreeRegressor()
            ]

In [ ]:
model_names = {
    type(SVR()).__name__: 'SVR',
    type(RandomForestRegressor()).__name__: 'RF',
    type(LinearRegression()).__name__: 'LR',
    type(Ridge()).__name__: 'RIDGE',
    type(GradientBoostingRegressor()).__name__: 'GBR',
    type(DecisionTreeRegressor()).__name__: 'DTR',
}

Parameter settings for MetaStream (considering a meta-level window size of 200)

In [ ]:
base_data_window = 672
base_delay_window = 0
base_sel_window_size = 24
meta_data_window = 200
threshold = 0.1

In [ ]:
pair_list = []
metastream_score = []
default_score = []

In [ ]:
meta_learner = RandomForestClassifier()
for pair in combinations(models, 2):
    reg1_name = type(pair[0]).__name__
    reg2_name = type(pair[1]).__name__
    print(f'Regressor pair: {reg1_name}, {reg2_name}')
    
    metas = MetaStream(meta_learner, pair, base_data_window, base_delay_window, base_sel_window_size, meta_data_window, strategy='combination', default=True, ensemble=True, pairs=True)
    metas.base_train(data=df, target='ArrDelay')
    metas.meta_train(data=df, target='ArrDelay')
    
    metas.print_results()
    base, meta = metas.get_results()
    
    pair_list.append(f'{model_names[reg1_name]}/{model_names[reg2_name]}')
    metastream_score.append(meta['recommended'])
    default_score.append(meta['default'])
    
    print('######################################################################')

In [ ]:
bar_width = 0.3
bar_locs = np.arange(len(pair_list))

plt.figure(figsize=(15,10))
plt.bar(bar_locs, metastream_score, width=bar_width, color='r', label='MetaStream')
plt.bar(bar_locs+bar_width, default_score, width=bar_width, color='b', label='Default')

plt.xticks(bar_locs+bar_width/2, pair_list, fontsize=14, rotation=90)
plt.xlabel('Regressor pairs', fontsize=20)

plt.yticks(fontsize=14)
plt.ylim(0, 1)
plt.ylabel('Meta-level error rate', fontsize=20)

plt.legend(loc='best', borderpad=1, fontsize=15)
plt.show()